In [1]:
import pandas as pd
import numpy as np

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',') #dtype={1:'str'}

population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

In [3]:
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
0,07/30/2020,07/21/2020,10/29/2020,VFC,Alameda,Alameda County,7.0,Closure Permanent
1,09/16/2020,04/27/2020,10/29/2020,Araca Merchandise LP.,San Fernando,Los Angeles County,89.0,Layoff Permanent
2,10/23/2020,09/25/2020,10/29/2020,"STAR Sports Theatre Arts & Recreation, Inc.",Culver City,Los Angeles County,60.0,Layoff Permanent
3,09/16/2020,04/25/2020,10/29/2020,Panasonic Avionics Corporation,Lake Forest,Orange County,582.0,Layoff Permanent
4,09/03/2020,05/08/2020,10/29/2020,Molson Coors Beverage Company,Baldwin Park,Los Angeles County,436.0,Closure Permanent


In [4]:
cali_layoffs.tail()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
9466,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No County Name,86.0,Layoff Permanent
9467,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No County Name,49.0,Closure Permanent
9468,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No County Name,11.0,Closure Permanent
9469,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No County Name,18.0,Layoff Unknown at this time
9470,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No County Name,50.0,Closure Permanent


## California 2020 Layoffs

We are selecting 2020 records by isolating the year on the `effective date` column.
By choosing `effective date` we make sure that these layoffs actually occurred in 2020, as some layoff notices can be issues much earlier the layoff date.

In [5]:
cali_layoffs['Year'] = cali_layoffs['Effective Date'].str[-4:]
cali_2020 = cali_layoffs[cali_layoffs['Year'] == '2020']
cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')
cali_2020.tail()

<ipython-input-5-59fdb45e4edc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5689,10/30/2019,01/03/2020,11/01/2019,"Ashford University, LLC",San Diego,San Diego County,218,Layoff Permanent,2020
5691,11/01/2019,01/10/2020,11/01/2019,American General Life Insurance Company,Pasadena,Los Angeles County,40,Closure Permanent,2020
5708,10/24/2019,01/01/2020,10/28/2019,"Hotel Del Coronado, Curio Collection By Hilton...",Coronado,San Diego County,160,Layoff Temporary,2020
5711,10/17/2019,05/24/2020,10/23/2019,Wells Fargo,Concord,Contra Costa County,357,Layoff Permanent,2020
5753,09/26/2019,01/16/2020,09/30/2019,"Forever 21 Logistics, LLC",Los Angeles,Los Angeles County,1170,Closure Permanent,2020


In [6]:
cali_2020['County'] = cali_2020['County'].str.strip()
cali_2020.loc[~(cali_2020['County'].str.endswith('County')),'County'] = cali_2020['County'] + " County"

<ipython-input-6-0b43f9e4a6a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['County'] = cali_2020['County'].str.strip()


In [7]:
cali_population.head(5)

,California,"39,512,223"
0,".Alameda County, California",1671329
1,".Alpine County, California",1129
2,".Amador County, California",39752
3,".Butte County, California",219186
4,".Calaveras County, California",45905


In [8]:
cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
cali_population['County Names'] = cali_population['Counties'].str[1:]
cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
cali_population.head(4)

,Counties,Population,County Names
0,".Alameda County, California",1671329,Alameda County
1,".Alpine County, California",1129,Alpine County
2,".Amador County, California",39752,Amador County
3,".Butte County, California",219186,Butte County


In [9]:
len(cali_2020)

5658

In [10]:
cali_merge = pd.merge(cali_2020, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge.drop(columns='Counties', inplace=True)
cali_merge.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year,Population,County Names
0,07/30/2020,07/21/2020,10/29/2020,VFC,Alameda,Alameda County,7,Closure Permanent,2020,1671329,Alameda County
1,09/16/2020,04/27/2020,10/29/2020,Araca Merchandise LP.,San Fernando,Los Angeles County,89,Layoff Permanent,2020,10039107,Los Angeles County
2,10/23/2020,09/25/2020,10/29/2020,"STAR Sports Theatre Arts & Recreation, Inc.",Culver City,Los Angeles County,60,Layoff Permanent,2020,10039107,Los Angeles County
3,09/16/2020,04/25/2020,10/29/2020,Panasonic Avionics Corporation,Lake Forest,Orange County,582,Layoff Permanent,2020,3175692,Orange County
4,09/03/2020,05/08/2020,10/29/2020,Molson Coors Beverage Company,Baldwin Park,Los Angeles County,436,Closure Permanent,2020,10039107,Los Angeles County


In [21]:
layoffs_by_county = cali_merge.groupby(['County Names','Year']).agg('sum')
layoffs = layoffs_by_county.merge(cali_population, how="left", on="County Names")
layoffs.drop(columns=['Counties', 'Population_x'], inplace=True)
layoffs.rename(columns={"Population_y":"Population"}, inplace=True)
layoffs['Percentage Laid Off'] = (layoffs['Employees']/layoffs['Population']) * 100
layoffs.sort_values(by='Percentage Laid Off', ascending=False)

,County Names,Employees,Population,Percentage Laid Off
18,Mariposa County,2262,17203,13.148869
33,San Francisco County,56652,881549,6.426415
21,Mono County,879,14444,6.085572
12,Inyo County,539,18039,2.987971
32,San Diego County,75458,3338330,2.260352
36,San Mateo County,16987,766573,2.215966
23,Napa County,2975,137744,2.159804
25,Orange County,64764,3175692,2.039367
0,Alameda County,32756,1671329,1.959877
15,Los Angeles County,191307,10039107,1.905618


## Next Steps

- [x] Standardize the county names
- [x] calculate percentage of eviction per county
- [ ] write up findings for Evictions-CA google doc
- [ ] Get city names and county names to match up the cities with 'No County Name' in County column (later for information prior to 2020 - which adds context)
- [ ] combine population information with fips codes to get the population for each city